This Markdown objective is to find the rent from the df column named infos. this column has many problems namely the enter is not the same. 

In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np


[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("fr_crous_logement_france_entiere.csv", sep = ';')
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,openinghours,interneturl,appointmenturl,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7796516418,2.0441513062",http://ephoto.nuonet.fr/link/Dz4FNVwrUWMAYAE5A...,Île-de-France
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"49.0482978821,2.02688694",http://stockcrous.fr/static-images/10bcd8bb8ec...,Île-de-France
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7906913757,2.3169708252",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.698802948,2.1578958035",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7015419006,2.1846776009",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,NaN,https://www.crous-normandie.fr/,NaN,NaN,NaN,NaN,"49.2032051086,-0.3392136097",https://admin-v2.crous-mobile.fr//media/crous-...,Normandie
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,NaN,https://www.crous-bfc.fr/,NaN,http://www.crous-bfc.fr/logements/logement-cro...,NaN,https://www.crous-bfc.fr/,"47.2365074158,6.0153431892",https://admin-v2.crous-mobile.fr//media/files/...,Bourgogne-Franche-Comté
785,335,Résidence Jean 

Part I : Initialisation pour détecter la présence de symbole dans les mots. 

On va créer une fonction qui prend en argument un mot et une liste qui va déctecter s il y a un des élèments de cette liste dans le mot. Cette fonction va ensuite dans notre fonction principale. 

In [3]:
"""
String_checking est une fonction qui prend un mot en argument et un élèment et retourne 0 si l'élèmenet n'est 
pas inclus dans le mot et 1 inversement. 
"""


def string_checking(string, check):
    result = 0 
    if string.find(check) == -1:
        result = 0
    else: 
        result = 1
    return result


In [4]:
"""
Cette fonction prend en argument un mot à tester et une liste. Elle va utiliser la fonction précédente pour
vérifier qu'il n'y a aucun élèment de la liste dans notre mot. De même elle retourne 0 ou 1 en fonction de
s'il y a un élèment ou non. 
"""

def word_ban(string, liste):
    result = 0 
    for check in liste: 
        if string_checking(string, check) == 1: 
            result = 1 
        else: 
            result = 0
    return result 
            

Part II : Main functions 

README 

On va créer une fonction pour trouver le loyer à partir de la fonction précèdente. La fonction prend pour argument un texte (ici df['infos'][j] qui correspond au texte informatif d'une résidence où se trouve le loyer. La fonction va nettoyer le texte d'une manière très précise : 

1 - On va enlever tout les espaces du texte de façon à n'avoir qu'un block uniforme de texte. 
2 - on va enlever les parenthèses (problèmes souvent eu - à voir si c'est toujours le cas sans) 
3 - On va enlever les petits 2 des mètres carrés qui posent souvent dans la suite du code. 
4 - On va remplacer les à en € de façon à avoir les loyers qui sont inscrits de la manière suivante : 200 à 350€ (notrer recherche se fonde grâce au €). 
5 - On va remplacer les 'euros' en €
6 - on va remplacer les €€ en € car il peut y avoir des '200€à350€' qui deviennent '200€€350€'et nous on veut '200€350€'. 
7 - On va rajouter un espace après chaque '€'. 
8 - On normalise les , en . pour avoir les mêmes typologies
9 - on enlever les : (à voir si sa fonctionne toujours sans). 

On va ensuite tokeniser notre block de texte grâce à la fonction word_tokenize du package nltk. On va ainsi passer de blocks de caractères séparé par un espace à une liste de mots. 

Exemple (df["infos"][0]): 

On passe de :

"Logements HLM

70 T1 : 325 euros
14 T1bis de 30m² : 373,50 euros
13 T2 de 37m² : 404 euros

   
   
   
 Photos de la résidence"
   
à : 

"['LogementsHLM', '70T1', '325€', '14T1bisde30m373.50€', '13T2de37m404€', 'Photosdelarésidence', 'https//www.flickr.com/photos/crousaixmarseilleavignon/albums/72157679829208245']"

L'idée va ensuite de trouver le loyer grâce à cette liste de mot. On va ainsi appliquer la fonction rent. cette fonction prend en argument cette liste de mots. Pour chaque élèments de notre liste on va d'abord chercher l'index de € (s'il existe et s'il n'appartient pas à la liste des mots interdits - on ne veut pas les moments des cautions par exemple). On va ensuite regarder si le caractère à la gauche du € est un chiffre ou non. Si c'est un chiffre on va réetirer cela pour le caractère à gauche de ce dernier. Ainsi de suite jusqu'à trouver un élèment qui n'est pas un chiffre. Dans ce cas là on va arrêter et prendre le nombre entre les deux indices. Le problème est celui des virgule : on rajoute une condition qui dit que si on trouve une virgule on regarde à gauche si c'est un chiffre. On obtient ainsi une liste des loyers avec quelques valeurs abberrantes que l'on gera après. 



In [5]:
"""
Liste servant à ne pas prendre le montant des APL ou des cautions. Voir dans la suite du code. 
"""

List_banned_word = ['APL', 'Apl', 'Caution', 'caution', 'apl', 'CAUTION', "Charge", "de", "charge", "Charges", "charge", "Dépôtdegarantie","wifi", "wifirst","abonnement", "Logementssoumis", "machineàlaver"]


In [6]:
def rent(word):
    Loyer = [] #On crée une liste qui stockera le loyer
    #try:
    for i in range(0,len(word)): #on regarde dans chaque block de caractère 
        index = word[i].find("€") #on cherche l'index de l'euros normalement pour tout les loyers il y est 
        search = 0 #il nous servira à arrêter la boucle while 
        if word_ban(word[i],List_banned_word) == 0 and word[i].find("€") != -1: #On regarde si dans le block de mot il n'y a pas un mot banni comme APL pour éviter d'avoir le montant de ceux-ci, ce qui fausserait nos résultats.
            while search == 0: #Tant que notre variable de contrôle est égale à 0, on va chercher le loyer (ie : tant que ce sont des chiffres, on va chercher la nature (chffre ou non) de ce qui se situe à gauche.
                search_index = word[i][index - 1] #On cherche l'index de ce qui se trouve à gauche de l'euros
                try: #Try permet de tester si c'est un chiffre tout en ne bloquant pas les opérations avec une erreure
                    search_index = int(search_index) #On regare si à gauche de l'ancien index c'est un entier ou non (Dans le cas contraire on sera dans except).
                    index = index - 1 #on remonte de 1 l'index pour voir si ce qui se trouve à gauche de notre chiffre est également un chiffre.
                except: #si ce n'est pas un entier à gauche de l'euros ou du dernier chiffre identifié. 
                    if word[i][index - 1] == ".": #on regarde si ce n'est pas un point comme dans 350.85 euros. Si on ne met pas cette ligne on aurait seulement 85 et non 350.85. 
                        index = index - 2 #On va regarde ce qui est avant le point pour trouver le reste du montant
                        search_index = word[i][index - 1] #Sysiphe remonte sa pierre et recommence le même processus qu'avant
                        try: #on regarde de nouveau si à gauche du point c'est un entier
                            search_index = int(search_index) 
                            index = index - 1 
                        except:
                            search = 1 #si ce n'est pas un chiffre ie une lettre on arrête
                            index = -1 #permet de ne pas avoir des erreurs du style .50
    
                    else: #si ce n'est pas un point on arrête le compte. 
                        search = 1
                #print(word[i][index : word[i].find("euros")])

            if word[i][index : word[i].find("€")]!= '': #on regarde le fait que le token ne soit pas juste un '€'
                Loyer.append(float(word[i][index : word[i].find("€")])) #On ajoute au loyer le loyer trouvé
    #except:
        #if word[i] != "€" :
            #List_banned_word.append(word[i])
            #print(List_banned_word)
            #rent(word)
    return Loyer

In [7]:
def rent_find(residence): 
    residence = residence.replace(' ', '')
    residence = residence.replace('²', '')
    residence = residence.replace('mà', '')
    residence = residence.replace('à', '€')
    residence = residence.replace('euros', '€')
    residence = residence.replace('€€', '€')
    residence = residence.replace('€', '€ ')
    residence = residence.replace(',', '.')
    word =  nltk.word_tokenize(residence, language='french') #on transforme notre scripts en block de mots. ceux qui nous intéressent ont un euros à la fin (mais il y en a qui ont un euros à la fin et qui ne nous intéresse pas
    return rent(word)
    

Part III : construction du Dataframe 

on crée une boucle for sur toute les résidences. si la fonction foctionne où l'ajoute à une liste sinon on ajoute NAN et on crée la colone loyer. 

In [8]:
a = 0
b = 0
Loyer = []
for j in range(0,df.shape[0]):
    try:
        Loyer.append(rent_find(df["infos"][j]))
        b = b + 1
    except: 
        Loyer.append(None)
        a = a + 1
df['Loyer'] = Loyer
print("échec pour ",a)
print("réussite pour ", b)
print("total ", a + b)

échec pour  11
réussite pour  777
total  788


In [9]:
"""
fonction servant à ne garder que les loyers et enlever les quelques erreures du style 5.99 qui serait l'abonnement au wifi
"""

def remove_fun(residence):
    for i in range(0, len(residence)):
        integer = int(residence[i]) #on transforme le loyer i de la résidence en un entier 
                                #pour ne pas avoir les chiffres derrières la virgule
        text = str(integer) #on transforme cet entier en texte
        if len(text) != 3: 
            residence.remove(residence[i]) #on enlève dès que c'est différent de 3 ie ce n'est pas un 
                                           #loyer qui pourrait être au Crous.                          
    return residence

    
               

In [14]:
"""
A lancer plusieurs fois - la fonction len à l'air de buguer sur python
"""

for j in range(0,df.shape[0]):
    if df["Loyer"][j] != None : #s'il y a un Loyer
        try : 
            remove_fun(df["Loyer"][j]) #on texte la fonction 
        except:  #On ressaye ainsi de suite
            try: 
                remove_fun(df["Loyer"][j])
            except: 
                remove_fun(df["Loyer"][j])
                try: 
                    remove_fun(df["Loyer"][j])
                except: 
                    remove_fun(df["Loyer"][j])

In [15]:
"""
On crée trois colonnes stockant le loyer maximum, le loyer minimum et la moyenne des loyers. 
"""

Max = []
Min = []
Mean = []
a = 0
for j in range(0,df.shape[0]):
    if df["Loyer"][j] != None and df["Loyer"][j] != []:
        a = a + 1
        Max.append(max(df["Loyer"][j]))
        Min.append(min(df["Loyer"][j]))
        Mean.append(np.mean(df["Loyer"][j]))
    else: 
         Max.append(np.NaN)
         Min.append(np.NaN)
         Mean.append(np.NaN)
df['Max Loyer'] = Max
df['Min Loyer'] = Min
df['Mean Loyer'] =  Mean
print(a)

482


In [16]:
df.head()

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7796516418,2.0441513062",http://ephoto.nuonet.fr/link/Dz4FNVwrUWMAYAE5A...,Île-de-France,"[333.0, 537.0]",537.0,333.0,435.0
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"49.0482978821,2.02688694",http://stockcrous.fr/static-images/10bcd8bb8ec...,Île-de-France,[334.0],334.0,334.0,334.0
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7906913757,2.3169708252",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France,"[332.0, 359.0]",359.0,332.0,345.5
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.698802948,2.1578958035",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France,[362.0],362.0,362.0,362.0
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7015419006,2.1846776009",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France,[154.0],154.0,154.0,154.0


In [17]:
print("Nombre de loyers manquant : ", df["Mean Loyer"].isna().sum())

Nombre de loyers manquant :  306


In [21]:
df.to_csv(r'Immobilier-etudiant-.csv', index = False) #pour stocker le nouveau dataframe 